# Create dummy sales data

In [ ]:
import random
from datetime import datetime
%pip install faker
from faker import Faker
from sqlalchemy import create_engine, Column, Integer, String, Numeric, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

host = ''
username = 'postgres'
password = ''
database = 'sales_db'

# Database configuration
DATABASE_URL = f'postgresql://{username}:{password}@{host}:5432/{database}'

# SqlAlchemy config
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()
fake = Faker()

# Models
class Customer(Base):
    __tablename__ = 'customers'
    customer_id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    phone = Column(String(20))
    created_at = Column(DateTime, default=datetime.utcnow)
    modified_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

class Product(Base):
    __tablename__ = 'products'
    product_id = Column(Integer, primary_key=True)
    product_name = Column(String(100), nullable=False)
    description = Column(String)
    price = Column(Numeric(10, 2), nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    modified_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

class Sale(Base):
    __tablename__ = 'sales'
    sale_id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey('customers.customer_id'), nullable=False)
    product_id = Column(Integer, ForeignKey('products.product_id'), nullable=False)
    quantity = Column(Integer, nullable=False)
    sale_date = Column(DateTime, default=datetime.utcnow)
    created_at = Column(DateTime, default=datetime.utcnow)
    modified_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    customer = relationship('Customer')
    product = relationship('Product')

# Create all the tables if not exists
Base.metadata.create_all(engine)

# Fake data for customers
def create_dummy_customers(n):
    customers = []
    for _ in range(n):
        customer = Customer(
            first_name=fake.first_name(),
            last_name=fake.last_name(),
            email=fake.unique.email(),
            phone=fake.phone_number()[:20]
        )
        customers.append(customer)
    session.add_all(customers)
    session.commit()

# Fake data for products
def create_dummy_products(n):
    products = []
    for _ in range(n):
        product = Product(
            product_name=fake.word(),
            description=fake.text(),
            price=round(random.uniform(10, 1000), 2)
        )
        products.append(product)
    session.add_all(products)
    session.commit()

# Fake data for sales
def create_dummy_sales(n):
    customer_ids = [customer.customer_id for customer in session.query(Customer).all()]
    product_ids = [product.product_id for product in session.query(Product).all()]
    
    sales = []
    for _ in range(n):
        sale = Sale(
            customer_id=random.choice(customer_ids),
            product_id=random.choice(product_ids),
            quantity=random.randint(1, 10)
        )
        sales.append(sale)
    session.add_all(sales)
    session.commit()


#create_dummy_customers(20)
#create_dummy_products(10)
create_dummy_sales(50)

print("Success data creation")
